<a href="https://colab.research.google.com/github/yeray142/AIML_project/blob/master/notebooks/SVM_CODE_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML4AI | SVM with "CODE" column & old vesrion

### Set up for Project

##### Library

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [2]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached optuna-3.0.4-py3-none-any.whl (348 kB)
  Using cached cmaes-0.9.0-py3-none-any.whl (23 kB)
  Using cached cliff-4.1.0-py3-none-any.whl (81 kB)
  Using cached colorlog-6.7.0-py2.py3-none-any.whl (11 kB)
  Using cached alembic-1.8.1-py3-none-any.whl (209 kB)
  Using cached Mako-1.2.4-py3-none-any.whl (78 kB)
  Using cached stevedore-4.1.1-py3-none-any.whl (50 kB)
  Using cached cmd2-2.4.2-py3-none-any.whl (147 kB)
  Using cached autopage-0.5.1-py3-none-any.whl (29 kB)
  Using cached pyperclip-1.8.2.tar.gz (20 kB)
     |████████████████████████████████| 112 kB 5.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=3f9082f784f8fde50c8ea09f35d6acf1a8077707379cb95e0152c3e043aacabe
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [3]:
# Modeling
import sklearn
assert sklearn.__version__ >= "0.20"

from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score, KFold,StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

from sklearn.svm import SVC
from sklearn.metrics import log_loss

In [4]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler

#### Data Load

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
df_train = pd.read_csv("/content/drive/MyDrive/ML4AI/cleaned_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/ML4AI/cleaned_test.csv")

In [8]:
df_train = df_train.drop(['Unnamed: 0'], axis=1)
df_test = df_test.drop(['index'],axis =1)

## Data EDA

#### **Basic EDA**

In [9]:
df_train.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,days_birth,days_employed,work_phone,home_phone,email,occup_type,family_size,begin_month,credit,CODE
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,13899,4709,0,0,0,No job,2.0,6,1.0,F-13899202500.0Commercial associate
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,11380,1540,0,0,1,Laborers,3.0,5,1.0,F-11380247500.0Commercial associate
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,19087,4434,0,1,0,Managers,2.0,22,2.0,M-19087450000.0Working
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,15088,2092,0,1,0,Sales staff,2.0,37,0.0,F-15088202500.0Commercial associate
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,15037,2105,0,0,0,Managers,2.0,26,2.0,F-15037157500.0State servant


In [10]:
df_test.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,days_birth,days_employed,work_phone,home_phone,email,occup_type,family_size,begin_month,CODE
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,21990,0,0,1,0,No job,2.0,60,M-21990112500.0Pensioner
1,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,18964,8671,0,1,0,Core staff,2.0,36,F-18964135000.0State servant
2,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,15887,217,1,1,0,Laborers,2.0,40,F-1588769372.0Working
3,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,19270,2531,1,0,0,Drivers,2.0,41,M-19270112500.0Commercial associate
4,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,17822,9385,1,0,0,Managers,2.0,8,F-17822225000.0State servant


In [11]:
df_train.describe()

,child_num,income_total,days_birth,days_employed,work_phone,home_phone,email,family_size,begin_month,credit
count,24817.000000,2.481700e+04,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000
mean,0.430874,1.868744e+05,15951.287625,2198.272112,0.225249,0.294274,0.091510,2.199057,26.054318,1.514325
std,0.750609,1.008656e+05,4205.102462,2371.033103,0.417754,0.455725,0.288339,0.919132,16.532853,0.704571
min,0.000000,2.700000e+04,7705.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,0.000000,1.215000e+05,12443.000000,407.000000,0.000000,0.000000,0.000000,2.000000,12.000000,1.000000
50%,0.000000,1.575000e+05,15536.000000,1539.000000,0.000000,0.000000,0.000000,2.000000,24.000000,2.000000
75%,1.000000,2.250000e+05,19428.000000,3153.000000,0.000000,1.000000,0.000000,3.000000,39.000000,2.000000
max,19.000000,1.575000e+06,25152.000000,15713.000000,1.000000,1.000000,1.000000,20.000000,60.000000,2.000000


## Data modeling

### Data Preprocessing for construct model

1) Encoding of categorical features

In [12]:
num_col = df_train.dtypes[df_train.dtypes != "object"].index.tolist()
num_col.remove('credit')

cat_col = df_train.dtypes[df_train.dtypes == "object"].index.tolist()

In [13]:
num_col

['child_num',
 'income_total',
 'days_birth',
 'days_employed',
 'work_phone',
 'home_phone',
 'email',
 'family_size',
 'begin_month']

In [14]:
cat_col

['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occup_type',
 'CODE']

In [15]:
# StandardScaler
scaler = StandardScaler()
df_train[num_col] = scaler.fit_transform(df_train[num_col])
df_test[num_col] = scaler.fit_transform(df_test[num_col])

In [16]:
# OrdinalEncoding
Encoder = OrdinalEncoder()
df_train[cat_col] = Encoder.fit_transform(df_train[cat_col], df_train['credit'])
df_test[cat_col] = Encoder.fit_transform(df_test[cat_col])


In [17]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24817 entries, 0 to 24816
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         24817 non-null  float64
 1   car            24817 non-null  float64
 2   reality        24817 non-null  float64
 3   child_num      24817 non-null  float64
 4   income_total   24817 non-null  float64
 5   income_type    24817 non-null  float64
 6   edu_type       24817 non-null  float64
 7   family_type    24817 non-null  float64
 8   house_type     24817 non-null  float64
 9   days_birth     24817 non-null  float64
 10  days_employed  24817 non-null  float64
 11  work_phone     24817 non-null  float64
 12  home_phone     24817 non-null  float64
 13  email          24817 non-null  float64
 14  occup_type     24817 non-null  float64
 15  family_size    24817 non-null  float64
 16  begin_month    24817 non-null  float64
 17  credit         24817 non-null  float64
 18  CODE  

3) Data Split ⇒ We need to split with stratified split

In [18]:
X_train, X_val, y_train, y_val = train_test_split(df_train.loc[:,df_train.columns != 'credit'], df_train['credit'], test_size = 0.2)

In [19]:
X_train.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,days_birth,days_employed,work_phone,home_phone,email,occup_type,family_size,begin_month,CODE
5180,0.0,1.0,0.0,-0.574044,-0.648147,4.0,4.0,1.0,1.0,-0.782229,-0.344275,1.854598,-0.64574,-0.317376,11.0,-0.216575,0.057201,1039.0
763,1.0,1.0,1.0,-0.574044,-0.291229,4.0,4.0,1.0,1.0,0.782568,-0.696872,-0.539200,-0.64574,-0.317376,8.0,-0.216575,-1.333996,7842.0
14128,1.0,0.0,0.0,-0.574044,0.377992,4.0,4.0,1.0,1.0,-1.616471,-0.483880,-0.539200,1.54861,-0.317376,8.0,-0.216575,-1.213022,8526.0
13275,0.0,0.0,0.0,-0.574044,0.824140,4.0,2.0,0.0,1.0,-1.704461,-0.809905,1.854598,-0.64574,3.150839,11.0,-0.216575,-0.305720,5465.0
7035,1.0,1.0,0.0,0.758235,-0.068155,4.0,4.0,1.0,1.0,-1.201252,-0.321500,-0.539200,-0.64574,-0.317376,15.0,0.871430,-0.366207,5934.0


In [20]:
y_train

5180     2.0
763      2.0
14128    2.0
13275    2.0
7035     1.0
        ... 
23554    1.0
19186    2.0
4342     2.0
14112    2.0
10314    0.0
Name: credit, Length: 19853, dtype: float64

### Modeling

- Optuna

In [21]:
def objective(trial: Trial) -> float:
    params_svc = {
        "random_state": 42,
        'kernel': 'rbf',
        'C' : trial.suggest_loguniform('C', 1e-5,1e+5),
        'gamma' : trial.suggest_loguniform('gamma', 1e-5,1e+5),
        'probability' : True,
    }
    model = sklearn.svm.SVC(**params_svc)
    model.fit(
        X_train,
        y_train
    )

    svc_pred = model.predict_proba(X_val)
    log_score = log_loss(y_val, svc_pred)
    
    return log_score

In [22]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="svc_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2022-12-11 09:57:30,864] A new study created in memory with name: svc_parameter_opt
[I 2022-12-11 10:05:24,043] Trial 0 finished with value: 0.8611161816232059 and parameters: {'C': 0.05564180225431373, 'gamma': 32147.193482816965}. Best is trial 0 with value: 0.8611161816232059.
[I 2022-12-11 10:17:21,039] Trial 1 finished with value: 0.864671934437308 and parameters: {'C': 208.90047049266641, 'gamma': 9.695826644515218}. Best is trial 0 with value: 0.8611161816232059.
[I 2022-12-11 10:19:26,889] Trial 2 finished with value: 0.8856729773720726 and parameters: {'C': 0.0003632339256943143, 'gamma': 0.0003630322466779861}. Best is trial 0 with value: 0.8611161816232059.
[I 2022-12-11 10:21:14,238] Trial 3 finished with value: 0.8746418959093488 and parameters: {'C': 3.809220577048033e-05, 'gamma': 4589.458612326462}. Best is trial 0 with value: 0.8611161816232059.
[I 2022-12-11 10:34:26,381] Trial 4 finished with value: 0.8829855716109769 and parameters: {'C': 10.260065124896771, 'gam

Best Score: 0.8242117455964451
Best trial: {'C': 0.011026112761510002, 'gamma': 1.7683340776662542}


In [23]:
study.best_params['C']

0.011026112761510002

In [24]:
study.best_params['gamma']

1.7683340776662542

--- 
K Fold

In [25]:
p_optuna = {
    'kernel' : 'rbf',
    "random_state": 42,
    'C' : study.best_params['C'],
    'gamma' : study.best_params['gamma'],
    'probability' : True,
}

In [26]:
def stratified_kfold_svc(p, n_fold, X, y, X_test):
    # Declaring Stratified K-Fold:
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    
    # Initializing arrays:
    svc_oof = np.zeros((X.shape[0], 3))
    svc_preds = np.zeros((X_test.shape[0], 3))

    # Main loop:
    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============")
        
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
        # Model fit:
        model = SVC(**p)
        model.fit(X_train, y_train)

        svc_oof[valid_idx] = model.predict_proba(X_valid) # Infers the class probability of train dataset
        svc_preds += model.predict_proba(X_test) / n_fold # Infers the class probability of test dataset
        print(f"Log Loss Score: {log_loss(y_valid, svc_oof[valid_idx]):.5f}")

    log_score = log_loss(y, svc_oof)
    print(f"\nLog Loss Score: {log_score:.5f}")

    return svc_oof, svc_preds

In [27]:
svc_oof_optuna, svc_preds_optuna = stratified_kfold_svc(p_optuna, 10, df_train.drop("credit", axis=1),df_train['credit'] , df_test)

============ Fold 0 ============
Log Loss Score: 0.82583
============ Fold 1 ============
Log Loss Score: 0.83474
============ Fold 2 ============
Log Loss Score: 0.83107
============ Fold 3 ============
Log Loss Score: 0.83133
============ Fold 4 ============
Log Loss Score: 0.82984
============ Fold 5 ============
Log Loss Score: 0.81810
============ Fold 6 ============
Log Loss Score: 0.83036
============ Fold 7 ============
Log Loss Score: 0.83660
============ Fold 8 ============
Log Loss Score: 0.82373
============ Fold 9 ============
Log Loss Score: 0.84221

Log Loss Score: 0.83038


In [28]:
p_grid = {
    'kernel' : 'rbf',
    "random_state": 42,
    'C' : 10.0,
    'gamma' : 0.1,
    'probability' : True,
}

In [29]:
svc_oof_grid, svc_preds_grid = stratified_kfold_svc(p_grid, 10, df_train.drop("credit", axis=1),df_train['credit'] , df_test)

============ Fold 0 ============
Log Loss Score: 0.80037
============ Fold 1 ============
Log Loss Score: 0.80738
============ Fold 2 ============
Log Loss Score: 0.81360
============ Fold 3 ============
Log Loss Score: 0.81003
============ Fold 4 ============
Log Loss Score: 0.80327
============ Fold 5 ============
Log Loss Score: 0.79444
============ Fold 6 ============
Log Loss Score: 0.80600
============ Fold 7 ============
Log Loss Score: 0.81110
============ Fold 8 ============
Log Loss Score: 0.80236
============ Fold 9 ============
Log Loss Score: 0.80253

Log Loss Score: 0.80511


In [30]:
df_submission = pd.read_csv('/content/drive/MyDrive/ML4AI/sample_submission.csv')

In [31]:
df_submission.set_index("index", inplace=True)
df_submission[:] = svc_preds_grid
df_submission.head(20)

,0,1,2
index,,,
26457,0.109105,0.234768,0.656127
26458,0.111683,0.231625,0.656693
26459,0.097373,0.225046,0.677581
26460,0.112515,0.234580,0.652905
26461,0.107526,0.248646,0.643827
26462,0.107760,0.221540,0.670700
26463,0.111954,0.234558,0.653488
26464,0.105407,0.221302,0.673292
26465,0.102974,0.230306,0.666720


In [32]:
df_submission.to_csv('/content/drive/MyDrive/ML4AI/SVC_CODE.csv',index=False)